DATA UPLOADING 

In [ ]:
# Importing Libraries
import pandas as pd
import os

In [ ]:
# Setting Folder Path and Initializing Empty Dictionary to Store DataFrames
folder_path = "E:/Documents/Data Analyst Roadmap/SQL/SQL Projects/Bellabeat Case Study/archive/mturkfitbit_export_4.12.16-5.12.16/Fitabase Data 4.12.16-5.12.16"
df = {}

In [ ]:
# For Loop to Download All CSV Diles Into Pandas DataFrames
for filename in os.listdir(folder_path):
    try:
        file_path = os.path.join(folder_path, filename)
        df_name = filename.replace(".csv", "")
        df[df_name] = pd.read_csv(file_path)
        print(f"Loaded: {filename} --> {df_name} (shape: {df[df_name].shape})")
    except:
        print(f"Failed to load: {filename}")

In [ ]:
# Showcase First 5 Rows of Data to Ensure Proper Upload
for name, data in df.items():
    print(f"\n{name} (first 5 rows):")
    display(data.head(5))

In [ ]:
# Checking Data Structure of Each DataFrame
for name, data in df.items():
    print(f"\n{name} (Data Structure):")
    display(data.dtypes)

GENERAL DATA CLEANING

In [ ]:
# Dropping Rows With All NA Values and Duplicates
for name, data in df.items():
    original_shape = data.shape
    data.dropna(axis = "index", how = "all", inplace = True, ignore_index = False)
    data.drop_duplicates(inplace = True)
    print(f"{name} -- Before Cleaning {original_shape} -> After cleaning: {data.shape}")

FILTERING DATASET TO MEET KPIs
- I will be using dailyActivity_merged, dailyCalories_merged, sleepDay_merged, weightLogInfo_merged, and minuteMETsNarrow_merged

In [ ]:
# Looking at dailyActivity 
dailyActivity = df["dailyActivity_merged"]

In [ ]:
# Dropping Redundant Columns From DataFrames 
dailyActivity.drop(['TotalDistance', 'TrackerDistance', 'VeryActiveDistance', 'ModeratelyActiveDistance', 'LightActiveDistance', 'SedentaryActiveDistance', 'LoggedActivitiesDistance'], 
                   axis = 1, inplace = True, errors = "ignore") # axis = 1 to drop columns, inplace = True to directly change the DataFrame 

# Setting ActivityDate to DateTime Format
dailyActivity["ActivityDate"] = pd.to_datetime(dailyActivity["ActivityDate"])

In [ ]:
# Check If Update Was Successful
dailyActivity

In [ ]:
# Looking at sleepDay_merged
sleepDay = df["sleepDay_merged"]

In [ ]:
# Dropping TotalSleepRecords
sleepDay.drop(["TotalSleepRecords"], axis = 1, inplace = True, errors = "ignore")

# Adding TotalHoursAsleep and TotalHoursInBed
sleepDay['TotalHoursAsleep'] = (sleepDay['TotalMinutesAsleep'] / 60).round(2)
sleepDay['TotalHoursInBed'] = (sleepDay['TotalTimeInBed'] / 60).round(2)


# Setting SleepDay to DateTime Format
sleepDay["SleepDay"] = pd.to_datetime(sleepDay["SleepDay"], format="%m/%d/%Y %I:%M:%S %p")

# Check to See if Update was Successful
sleepDay

In [ ]:
# Looking at dailyCalories_merged
dailyCalories = df["dailyCalories_merged"]

In [ ]:
# Setting ActivityDay to DateTime Format
dailyCalories["ActivityDay"] = pd.to_datetime(dailyCalories["ActivityDay"])

# Check if Changes Were Successful
dailyCalories

In [ ]:
# Looking at weightLogInfo_merged
weightLogInfo = df["weightLogInfo_merged"]

# Dropping Fat, IsManualReport, LogId Columns as they Do Not Provide Insightful Data For Our Usecase
weightLogInfo.drop(["Fat", "IsManualReport", "LogId"], axis = 1, inplace = True, errors = "ignore")

# Round WegithKg, WeightPounds, and BMI to 2 Decimals For Cleaner Data
weightLogInfo[["WeightKg", "WeightPounds", "BMI"]] = weightLogInfo[["WeightKg", "WeightPounds", "BMI"]].round(2)

# Change Date to DateTime Format
weightLogInfo["Date"] = pd.to_datetime(weightLogInfo["Date"], format="%m/%d/%Y %I:%M:%S %p")
weightLogInfo["Date"] = weightLogInfo["Date"].dt.date

# Check if Changes Were Successful
weightLogInfo

In [ ]:
# Looking at minuteMETsNarrow_merged -- Will be used for a heatmap categorized by day and the time of day
METS = df["minuteMETsNarrow_merged"].copy(deep=True)

# Convert ActivityMinute to DateTime
METS["ActivityMinute"] = pd.to_datetime(METS["ActivityMinute"], format="%m/%d/%Y %I:%M:%S %p")

# Add a New Column Called "Day" to Show What Day of the Week the Date is
METS["Day"] = METS["ActivityMinute"].dt.day_name()

# Add a New Column Called "Hour" to Show What Hour (24 Hour Clock) the METs was Tracked
METS["Hour"] = METS["ActivityMinute"].dt.hour

# Convert "Activity Minute" to Only Include the Date and Not the Hour
METS["ActivityMinute"] = METS["ActivityMinute"].dt.date

# Reorder columns and Rename "ActivityMinute" to "Date"
METS = METS[["Id", "ActivityMinute", "Day", "Hour", "METs"]]
METS = METS.rename(columns = {"ActivityMinute":"Date"})

# Check to see if the changes were successful
METS

In [ ]:
# Looking at minuteMETsNarrow_merged -- This time used for finding AVG METs per time of day
METS_avg = df["minuteMETsNarrow_merged"].copy(deep=True)

# Convert ActivityMinute to DateTime
METS_avg["ActivityMinute"] = pd.to_datetime(METS_avg["ActivityMinute"], format="%m/%d/%Y %I:%M:%S %p")

# Convert ActivityMinute to Hours
METS_avg['ActivityMinute'] = METS_avg['ActivityMinute'].dt.hour

# Dropping Id, ActivityMinute, and Day
METS_avg = METS_avg.drop(['Id'], axis = 1)

# Renaming ActivityMinute column to Hour of Day
METS_avg = METS_avg.rename(columns={'ActivityMinute':'Hour_of_Day'})

# Grouping By Day, Finding the Average Value of METs
METS_avg = METS_avg.groupby(['Hour_of_Day']).agg({'METs':'mean'})
METS_avg["METs"] = METS_avg["METs"].round(2) # Rounding to two decimals

# Display
METS_avg 

UPLOAD RELEVANT DATAFRAMES INTO CSV FILES 

In [ ]:
# Setting a List to Iterate Over a For Loop
dataframes = [dailyActivity, dailyCalories, sleepDay, weightLogInfo, METS, METS_avg]
names = ['dailyActivity', 'dailyCalories', 'sleepDay', 'weightLogInfo', 'METS_3', 'METS_avg']

for i, name in zip(dataframes, names):
    i.to_csv(f"{name}.csv", index=False)